## Model Data using Cassandra


### Please just submit this notebook in the Submission and make sure each cell has been executed and output is clearly displayed.

### The aim of the project is to solve the three queries given below.

### Introduction

There is a music streaming app called SoundCloud, that has been using their music streaming app and collecting data on songs and user activity and their aim is to analyze this data especially understanding what songs users are listening to. Currently, they are not making use of a NoSQL db and they have the data stored as a CSV file, thus its difficult for them to query the data. So our task is to create a NoSQL database for helping them with the analysis.

#### Import Packages 

In [21]:
import pandas as pd
import numpy as np
import cassandra
import csv


##  The image below is a screenshot of what the data appears like in the event_data.csv

<img src="event_data_image.jpg">

#### Creating a Cluster

In [23]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

#### Create & Set Keyspace

In [24]:

# Create a keyspace with the CQL query
create_keyspace_query = """
CREATE KEYSPACE IF NOT EXISTS mykeyspace
WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 3};
"""
session.execute(create_keyspace_query)
print("Keyspace created successfully.")

# Set the keyspace for the session
session.set_keyspace('mykeyspace')
print("Keyspace set successfully for the session.")

Keyspace created successfully.
Keyspace set successfully for the session.


## List of Queries 

### 1. Find the artist_name, song_title and length of song the SoundCloud app history that was heard during  session_number = 338, and item_in_session_number  = 4


### 2. Find the artist_name, song_title (sorted by item_in_session_number) and name(fname and lname) of the user for user_id = 10, session_number = 182
    

### 3. Find every name(fname and lname) of the user from the SoundCloud app history that listened to the song_title 'All Hands Against His Own'




### Query1 Table1: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key

## Query 1

In [25]:
create_table_query = """
CREATE TABLE IF NOT EXISTS song_play_history (
    session_number int,
    item_in_session_number int,
    artist_name text,
    song_title text,
    song_length float,
    PRIMARY KEY (session_number, item_in_session_number)
);
"""
session.execute(create_table_query)

In [33]:
create_song_history_table_query = """
CREATE TABLE IF NOT EXISTS song_playback_history (
    session_id int,
    item_in_session int,
    artist text,
    title text,
    length float,
    PRIMARY KEY (session_id, item_in_session)
);
"""
session.execute(create_song_history_table_query)

### Let's insert our data into of table

In [37]:
file_name = 'event_data.csv'
with open(file_name, encoding='utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader)  # Skip the header
    for row in csv_reader:
        insert_query = """
        INSERT INTO song_play_history (session_number, item_in_session_number, artist_name, song_title, song_length)
        VALUES (%s, %s, %s, %s, %s)
        """
        # Adjusting indices based on the CSV structure provided
        try:
            session.execute(insert_query, (int(row[8]), int(row[3]), row[0], row[9], float(row[5])))
        except ValueError as e:
            print(f"Error processing row {row}: {e}")

### Validate our Data Model using a SELECT

In [38]:
select_query = """
SELECT artist_name, song_title, song_length
FROM song_play_history
WHERE session_number = 338 AND item_in_session_number = 4;
"""
rows = session.execute(select_query)
for row in rows:
    print(row.artist_name, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query2 Table2: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key 

In [39]:
create_table_query = """
CREATE TABLE IF NOT EXISTS user_session_songs (
    user_id int,
    session_number int,
    item_in_session_number int,
    artist_name text,
    song_title text,
    user_fname text,
    user_lname text,
    PRIMARY KEY ((user_id, session_number), item_in_session_number)
) WITH CLUSTERING ORDER BY (item_in_session_number ASC);
"""
session.execute(create_table_query)

### Let's insert our data into of table

In [40]:
file_name = 'event_data.csv'

with open(file_name, encoding='utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader)  # Fix typo here to correctly skip the header
    
    for row in csv_reader:
        query = """
        INSERT INTO user_session_songs (user_id, session_number, item_in_session_number, artist_name, song_title, user_fname, user_lname)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        # Adjust indices according to your CSV. Example uses placeholders.
        session.execute(query, (int(row[10]), int(row[8]), int(row[3]), row[0], row[9], row[1], row[4]))

### Validate our Data Model using a SELECT

In [11]:
query = """
SELECT artist_name, song_title, user_fname, user_lname
FROM user_session_songs
WHERE user_id = 10 AND session_number = 182
"""

try:
    rows = session.execute(query)
    for row in rows:
        print(f"Artist Name: {row.artist_name}, Song Title: {row.song_title}, User Name: {row.user_fname} {row.user_lname}")
except Exception as e:
    print(f"An error occurred: {e}")

Artist Name: Down To The Bone, Song Title: Keep On Keepin' On, User Name: Sylvie Cruz
Artist Name: Three Drives, Song Title: Greece 2000, User Name: Sylvie Cruz
Artist Name: Sebastien Tellier, Song Title: Kilometer, User Name: Sylvie Cruz
Artist Name: Lonnie Gordon, Song Title: Catch You Baby (Steve Pitron & Max Sanna Radio Edit), User Name: Sylvie Cruz


### Query3 Table3: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key

In [12]:
create_table_query = """
CREATE TABLE IF NOT EXISTS song_listened_by_user (
    song_title text,
    user_id int,
    user_fname text,
    user_lname text,
    PRIMARY KEY (song_title, user_id)
);
"""
session.execute(create_table_query)

### Let's insert our data into of table

In [13]:
file_name = 'event_data.csv'

# Correcting the typo in 'csvreader' to 'csv_reader' and ensuring proper variable usage
with open(file_name, encoding='utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader)  # Skip the header in the CSV file
    for row in csv_reader:
        query = """
        INSERT INTO song_listened_by_user (song_title, user_id, user_fname, user_lname)
        VALUES (%s, %s, %s, %s)
        """
        # Replace the row indices with the correct ones according to your CSV structure
        # Assuming the columns for song_title, user_id, user_fname, and user_lname are in correct order
        session.execute(query, (row[9], int(row[10]), row[1], row[4]))

### Validate our Data Model using a SELECT

In [14]:
query = "SELECT user_fname, user_lname FROM song_listened_by_user WHERE song_title = 'All Hands Against His Own';"
rows = session.execute(query)
for row in rows:
    print(f"{row.user_fname} {row.user_lname}")

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [15]:
try:
    session.execute("DROP TABLE IF EXISTS song_play_history;")
    print("Table 'song_play_history' dropped successfully.")
except Exception as e:
    print(f"Error dropping table 'song_play_history': {e}")

try:
    session.execute("DROP TABLE IF EXISTS user_session_songs;")
    print("Table 'user_session_songs' dropped successfully.")
except Exception as e:
    print(f"Error dropping table 'user_session_songs': {e}")

try:
    session.execute("DROP TABLE IF EXISTS song_listened_by_user;")
    print("Table 'song_listened_by_user' dropped successfully.")
except Exception as e:
    print(f"Error dropping table 'song_listened_by_user': {e}")

Table 'song_play_history' dropped successfully.
Table 'user_session_songs' dropped successfully.
Table 'song_listened_by_user' dropped successfully.


### Close the session and cluster connection¶

In [16]:
session.shutdown()
cluster.shutdown()